# Interpolate $\mathsf{\delta^{18}O}$ Initial field in NEMO

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import sys
sys.path.insert(0, '_libs/')

In [3]:
import glob
#import pyproj
import scipy.io
import numpy as np
import netCDF4 as nc
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap, addcyclic
% matplotlib inline

In [4]:
from NEMO_tools import int_profile, bin_profile, find_inland, reporj_NEMOgrid

In [5]:
from pykrige.ok import OrdinaryKriging
import pykrige.kriging_tools as kt

In [ ]:
MAT = scipy.io.loadmat('_data/Exchange/Temp_d18O_int.mat')
d18O_ini_xy = MAT['d18O_ini_xy'][:]
nav_lat = MAT['nav_lat'][:]
nav_lon = MAT['nav_lon'][:]

In [ ]:
lon_list = np.linspace(-180, 180, 360)
lat_list = np.linspace(60, 90, 30)
lonxy, latxy = np.meshgrid(lon_list, lat_list)

In [ ]:
num_layer = 31
curent_layer = 27

In [ ]:
d18O_ini_orca = np.empty([num_layer, np.size(nav_lon, 0), np.size(nav_lon, 1)])

In [ ]:
for i in range(curent_layer):
    int_temp_orca = reporj_NEMOgrid(lonxy, latxy, d18O_ini_xy[:, :, i], nav_lon, nav_lat, method='linear')
    d18O_ini_orca[i, :, :] = int_temp_orca

In [ ]:
for i in range(curent_layer, num_layer):
    d18O_ini_orca[i, :, :] = d18O_ini_orca[curent_layer-1, :, :]

In [ ]:
d18O_ini_masked = np.ma.masked_where(np.isnan(d18O_ini_orca), d18O_ini_orca)

In [ ]:
ETOPO2_Arctic=scipy.io.loadmat('_libs/ETOPO2_Arctic.mat')
lon_arctic=ETOPO2_Arctic['lon_arctic']
lat_arctic=ETOPO2_Arctic['lat_arctic']
topo_arctic=ETOPO2_Arctic['topo_arctic']

In [ ]:
# Adjust resolution
res_unit=5
lon_arctic=lon_arctic[0:-1:res_unit, 0:-1:res_unit]
lat_arctic=lat_arctic[0:-1:res_unit, 0:-1:res_unit]
topo_arctic=topo_arctic[0:-1:res_unit, 0:-1:res_unit]*-1

In [ ]:
clevs=[1000, 2000, 3000]
for num in range(2):
    # Figures
    fig=plt.figure(figsize=(10, 10)); ax=plt.gca()
    proj=Basemap(projection='npstere', resolution='l', boundinglat=55, lon_0=90, round=True, ax=ax)
    proj.drawmeridians(np.arange(0, 360, 60), labels=[1, 1, 1, 1], fontsize=10, latmax=90,linewidth=0)
    proj.fillcontinents(color=[0.5, 0.5, 0.5], lake_color=None)
    proj.drawcoastlines(linewidth=1.5, color='k')
    gridx, gridy = proj(nav_lon, nav_lat)
    topox, topoy = proj(lon_arctic, lat_arctic)
    CS = proj.pcolor(gridx, gridy, d18O_ini_masked[num, :, :], vmin=30, vmax=100, cmap=plt.cm.gist_ncar_r)
    CS2 = proj.contour(topox, topoy, topo_arctic, clevs, linestyles='-', linewidths=1, colors=('k',), alpha=0.75)
    CS2.collections[0].set_label('1, 2, 3 km bathymetry')
    proj.plot(gridx.T, gridy.T, 'k-', lw=0.25)
    proj.plot(gridx, gridy, 'k-', lw=0.25)
    cbaxes = fig.add_axes([0.975, 0.15, 0.02, 0.5]) 
    CBar = plt.colorbar(CS, ax=ax, cax=cbaxes) 
    CBar.set_label('Interpolated d18O', fontsize=12)
    CBar.ax.tick_params(axis='y', length=0)
    ax.set_title('Initial field in a regular lat/lon frame', fontsize=15, y=1.025)

In [ ]:
# Save
save_var = { 'd18O_ini_orca': d18O_ini_orca, 'nav_lon': nav_lon, 'nav_lat': nav_lat}
scipy.io.savemat('_data/Exchange/NEMO_ORCA2_d18O.mat', mdict=save_var)